# Tutorial: Variables (Stateful `evaluate(env)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/variable.html

This is the stateful variant: definitions are kept in a shared environment across cells.


In [18]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import EvaluationEnvironment, evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")

env = EvaluationEnvironment()
stateful = evaluate(env)


Using bqn_jax from: /home/johtok/repos/jax/xlapl/bqn-jax/src/bqn_jax/__init__.py


## Before You Start

1. Run the setup cell first to create `env` and `stateful`.
2. Run cells in order from top to bottom, because later cells may depend on earlier definitions.
3. Each `stateful("...")` call updates and reuses the same environment.
4. If results look odd after reruns, restart the kernel and run all cells again.
5. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## Examples

Quick warm-up examples before the deeper sections.


In [19]:
stateful('hey ← "Hi there"')
stateful('hey ∾ ", World!"')


Array([ 72, 105,  32, 116, 104, 101, 114, 101,  44,  32,  87, 111, 114,
       108, 100,  33], dtype=int32)

## Defining variables

Define names with `←`, then update existing names with `↩` when needed.


In [20]:
stateful('pi‿e‿ten ← ⟨ π, ⋆1, 10 ⟩')
stateful('ten × pi')
stateful('three ⋈ ten - three ← 3')


Array([3., 7.], dtype=float32)

In [21]:
stateful('three ← 4')


Array(4., dtype=float32, weak_type=True)

In [22]:
stateful('three ↩ 4')
stateful('three = 3   # Wait why did I do that')
stateful('3 = three ↩ 3')
stateful("four ↩ 3    # four isn't defined yet")


NameError: Cannot update undefined name 'four'

## Variable roles

Identifier spelling and casing affect syntactic role in BQN.


In [ ]:
stateful('BQN ← "[many pages of specification]"')


Array([ 91, 109,  97, 110, 121,  32, 112,  97, 103, 101, 115,  32, 111,
       102,  32, 115, 112, 101,  99, 105, 102, 105,  99,  97, 116, 105,
       111, 110,  93], dtype=int32)

In [ ]:
stateful('three')
stateful('thrEe')
stateful('ThReE')
stateful('thr_EE')
stateful('__three')
stateful('_T_H_R_E_E_')


Array(3., dtype=float32, weak_type=True)

In [ ]:
stateful('- Three')
stateful('- _three')


Array(-3., dtype=float32, weak_type=True)

In [ ]:
stateful('1_000_000')


Array(1.e+06, dtype=float32, weak_type=True)

## Function assignment

Functions are first-class values, so assignment works for function definitions too.


In [ ]:
stateful('Base2 ← +⟜(2⊸×)´∘⌽')
stateful('Base2 1‿0‿1‿0')
stateful('Base2 "01010001"-\'0\'')
stateful('@ + Base2¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


Array([66., 81., 78.], dtype=float32)

In [ ]:
stateful('Base2')
stateful('base2 ↩ 16   # Change it to a number')
stateful('Base2')
stateful('Base2 6')


Array(16., dtype=float32, weak_type=True)

## Modifying part of an array

These examples show targeted updates through functional transforms like Under.


In [ ]:
stateful('"BQN"            # A list of characters')
stateful('-⟜1⌾(2⊸⊑) "BQN"  # Wait why did I do that')


Array(77., dtype=float32)

In [ ]:
stateful('(↕3) ⋈¨ "BQN"')
stateful('1 ⊑ "BQN"')


Array(81, dtype=int32)

In [ ]:
stateful('8⌾⊑ "BQN"        # Change the first element to 8')


Array(8., dtype=float32, weak_type=True)

In [ ]:
stateful('↕7')
stateful('4 ↑ ↕7           # The first four elements')
stateful('⌽⌾(4⊸↑) ↕7       # And reverse them')
stateful('⌽⌾(¯4⊸↑) ↕7      # Or reverse the last four')


Array([6, 5, 4, 3], dtype=int32)

In [ ]:
stateful('2⊸⌽⌾(1⊸⊑) "xyz"‿"ABCDE"‿"wxyz"‿"yz"')
stateful('2⊸⌽⌾(2⊸↓) "XYabcde"')


Array([101,  97,  98], dtype=int32)

In [ ]:
stateful('¯3 ↓ "abcdefgh"')
stateful('2 ↑ 4 ↓ "abcdefgh"')


Array([101, 102], dtype=int32)

In [ ]:
stateful('(\'A\'-\'a\')⊸+ ⌾ (2 ↑ 4⊸↓)  "abcdefgh"')


Array([0, 0], dtype=int32)

## Identity functions

Identity-like forms are useful when controlling data flow in larger expressions.


In [ ]:
stateful('⊢ "only"')
stateful('⊣ "only"')
stateful('"left" ⊢ "right"')
stateful('"left" ⊣ "right"')


Array([108, 101, 102, 116], dtype=int32)

## Modified assignment

Combine assignment with transforms to update values concisely.


In [23]:
stateful("a ← 4            # First it's a number\na")
stateful("a ↩ 4‿5‿6        # Now it's a list!\na")


Array([4., 5., 6.], dtype=float32, weak_type=True)

In [24]:
stateful('a ↩ a - 1\na')
stateful('a -↩ 1')


Array([2., 3., 4.], dtype=float32, weak_type=True)

In [25]:
stateful('a ∾˜↩ 0‿1')


Array([0., 1., 2., 3., 4.], dtype=float32, weak_type=True)

In [26]:
stateful('"abcd" ⌽∘⊣ "wxyz"')
stateful('a ⌽∘⊣↩ @')


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [27]:
stateful('a ⌽↩')
stateful('a 4⊸-↩           # And back again')


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [28]:
stateful('-⟜4⌾(¯2⊸↑) a')
stateful("a                # It hasn't changed, of course")


Array([4., 3., 2., 1., 0.], dtype=float32, weak_type=True)

In [29]:
stateful('a -⟜4⌾(¯2⊸↑)↩')


Array([-3., -4.], dtype=float32, weak_type=True)